<a href="https://colab.research.google.com/github/dldbwls00/NLP-sentiment-analysis-project/blob/main/kobert_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#필요 패키지 설치
!pip install mxnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 18.3 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1


In [4]:
!pip install gluonnlp pandas tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=661779 sha256=2e101c7cce632290dc65849c414732a3e1794c8b5e2c93ca142146860a893f46
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp


In [5]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.4 MB/s eta 0:00:00


In [6]:
!pip install tokenizers

In [7]:
!pip install torch

In [8]:
!pip install pandas

In [9]:
# !pip install transformers
# !pip install transformers==3.0.2

Building wheels for collected packages: tokenizers
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Building wheel for tokenizers (pyproject.toml) ... error
  ERROR: Failed building wheel for tokenizers
Failed to build tokenizers
ERROR: Could not build wheels for tokenizers, which is required to install pyproject.toml-based projects

해결 - https://github.com/googlecolab/colabtools/issues/3739

In [10]:
!python --version

Python 3.10.12


In [11]:
!pip install transformers==4.30.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [12]:
#KoBERT 깃허브에서 불러오기
#!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

ERROR: Could not find a version that satisfies the requirement onnxruntime<=1.8.0,==1.8.0 (from kobert) (from versions: 1.12.0, 1.12.1, 1.13.1, 1.14.0, 1.14.1, 1.15.0, 1.15.1, 1.16.0, 1.16.1, 1.16.2, 1.16.3)
ERROR: No matching distribution found for onnxruntime<=1.8.0,==1.8.0

해결 - https://github.com/SKTBrain/KoBERT/issues/102

In [13]:
#KoBERT 깃허브에서 불러오기
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-zsgw38t3/kobert-tokenizer_57d450ee81684526b65b733d2d6b4f1a
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-zsgw38t3/kobert-tokenizer_57d450ee81684526b65b733d2d6b4f1a
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=e10a3ff61e64b99c285df27a16a57e5b6b5d73942b003ff7c9ebb62978e4cb4a
  Stored in directory: /tmp/pip-ephem-wheel-cache-z_e0_q8y/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [14]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

In [15]:
#KoBERT
#from kobert.utils import get_tokenizer
#from kobert.pytorch_kobert import get_pytorch_kobert_model

ModuleNotFoundError: No module named 'kobert'

colab python3.10 업데이트 버그 - 하단 링크 참고
- https://blog.naver.com/newyearchive/223097878715
- https://github.com/SKTBrain/KoBERT/issues/104


kobert를 불러오는 대신 hugging face를 불러오고 trasnformer.py 내의 클래스를 colab cell에서 수정
- 기존 코드대로 실행하면 input_ids 가 dataset 길이만큼 전부 같은 형식으로 바뀌어 정확도가 매우 낮아짐
- tokenizer.tokenize로 변환시켜서 넣지 말고  BERTSentenceTransform 내부에서 직접 코드 수정 -
`#tokens_a = self._tokenizer(text_a)
tokens_a = self._tokenizer.tokenize(text_a) #수정`


여기에 정리 잘 돼 있음 >> https://complexoftaste.tistory.com/2

In [16]:
#KoBERT - hugging face 이용
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
#transformer
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [17]:
#GPU 설정
device = torch.device("cuda:0")

In [18]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [19]:
class BERTSentenceTransform:
    r"""BERT style data transformation.

    Parameters
    ----------
    tokenizer : BERTTokenizer.
        Tokenizer for the sentences.
    max_seq_length : int.
        Maximum sequence length of the sentences.
    pad : bool, default True
        Whether to pad the sentences to maximum length.
    pair : bool, default True
        Whether to transform sentences or sentence pairs.
    """

    def __init__(self, tokenizer, max_seq_length,vocab, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = vocab

    def __call__(self, line):
        """Perform transformation for sequence pairs or single sequences.

        The transformation is processed in the following steps:
        - tokenize the input sequences
        - insert [CLS], [SEP] as necessary
        - generate type ids to indicate whether a token belongs to the first
        sequence or the second sequence.
        - generate valid length

        For sequence pairs, the input is a tuple of 2 strings:
        text_a, text_b.

        Inputs:
            text_a: 'is this jacksonville ?'
            text_b: 'no it is not'
        Tokenization:
            text_a: 'is this jack ##son ##ville ?'
            text_b: 'no it is not .'
        Processed:
            tokens: '[CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]'
            type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
            valid_length: 14

        For single sequences, the input is a tuple of single string:
        text_a.

        Inputs:
            text_a: 'the dog is hairy .'
        Tokenization:
            text_a: 'the dog is hairy .'
        Processed:
            text_a: '[CLS] the dog is hairy . [SEP]'
            type_ids: 0     0   0   0  0     0 0
            valid_length: 7

        Parameters
        ----------
        line: tuple of str
            Input strings. For sequence pairs, the input is a tuple of 2 strings:
            (text_a, text_b). For single sequences, the input is a tuple of single
            string: (text_a,).

        Returns
        -------
        np.array: input token ids in 'int32', shape (batch_size, seq_length)
        np.array: valid length in 'int32', shape (batch_size,)
        np.array: input token type ids in 'int32', shape (batch_size, seq_length)

        """

        # convert to unicode
        text_a = line[0]
        if self._pair:
            assert len(line) == 2
            text_b = line[1]

        tokens_a = self._tokenizer.tokenize(text_a)
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer(text_b)

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        # The embedding vectors for `type=0` and `type=1` were learned during
        # pre-training and are added to the wordpiece embedding vector
        # (and position vector). This is not *strictly* necessary since
        # the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.

        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        #vocab = self._tokenizer.vocab
        vocab = self._vocab
        tokens = []
        tokens.append(vocab.cls_token)
        tokens.extend(tokens_a)
        tokens.append(vocab.sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(vocab.sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)

        # The valid length of sentences. Only real  tokens are attended to.
        valid_length = len(input_ids)

        if self._pad:
            # Zero-pad up to the sequence length.
            padding_length = self._max_seq_length - valid_length
            # use padding tokens for the rest
            input_ids.extend([vocab[vocab.padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
            np.array(segment_ids, dtype='int32')

In [20]:
"""class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        #transform = nlp.data.BERTSentenceTransform(
        #    tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))
"""
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [21]:
# google drive - colab 연결
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


감성대화말뭉치 training set + 사람문장1 이용
https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&dataSetSn=86

In [22]:
# 데이터셋 불러오기
import pandas as pd
conv_training = pd.read_excel('/content/drive/MyDrive/dataset/conv_training.xlsx')
conv_validation = pd.read_excel('/content/drive/MyDrive/dataset/conv_validation.xlsx')

In [23]:
# 데이터셋 잘 가져왔는지 확인
print(len(conv_training))
#print(len(conv_validation))
conv_training.sample(n=5)
#conv_validation.sample(n=5)

51630


,Unnamed: 0,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템문장1,사람문장2,시스템문장2,사람문장3,시스템문장3
14713,50855,청소년,남성,가족관계,해당없음,당황,외로운,오늘 학교에서 같은 반 애가 우리 부모님 이혼이 나 때문이냐고 물었어. 정말 그랬을까?,부모님 이혼 이유 때문에 죄책감을 느끼고 계시는군요.,정말 나 때문에 부모님이 이혼한 거면 어쩌지?,지금의 기분에서 벗어나기 위해 무언가 할 수 있는 것이 있을까요?,혼자 고민하지 말고 엄마한테 물어볼까? 그렇지만 어떤 대답일지 너무 두려워.,주위에서 지금의 감정을 나눌 수 있는 사람이 있었으면 좋겠어요.
8300,14661,중년,여성,"대인관계(부부, 자녀)",해당없음,당황,한심한,남편은 살 빼고 싶어서 운동한다면서 만날 입으로만 해.,남편이 운동을 안 하시나 봐요. 걱정되시겠어요.,조금만 빼면 건강해질 텐데 왜 만날 말만 하는지 걱정이야.,많이 걱정되시겠어요. 어떻게 하는 게 좋은 방법일까요?,운동을 할 수 있도록 회원권을 발급해주려고 해. 그러면 앞으로 정말 운동을 하지 않을까?,운동을 하게 해주셨군요. 생각하신 대로 잘 해결이 되었으면 좋겠어요.
10835,30688,노년,여성,재정,만성질환 무,당황,열등감,내가 지금까지 이거 밖에 못 모았다니.,많이 모으신 거 아닌가요? 그렇게 생각하시는 이유가 있나요?,주변은 돈도 많이 모아둬서 여행 다니던데 나는 은퇴하고 또 일하게 생겼어.,주변과 비교되어 열등감이 느껴지시는군요.,NaN,NaN
5220,32419,청년,여성,대인관계,해당없음,기쁨,신뢰하는,골치아픈 일이 있었는데 팀장님이 깔끔하게 처리하셨어.,곤란한 일을 책임자가 떠맡아 해결해 주시다니 든든하시곘어요.,팀장님이 숙련된 일처리 능력을 보여주셔서 존경스러웠어.,손아래 직원에게 신임을 얻는 상사가 무척 존경스러우시겠어요.,NaN,NaN
32879,49079,청소년,여성,학업 및 진로,해당없음,불안,회의적인,이번 시험에도 낮은 점수를 받았어. 노력한 만큼 성적이 나오지 않아서 많이 허무해.,노력한 만큼 결과가 나오지 않아 많이 허무하고 속상하시겠어요.,이래서 취업은 할 수 있을까? 이제 자신도 없고 뭘 해야 할지도 모르겠어.,이 상황에서 할 수 있는 것은 어떤 것이 있을까요?,아무래도 틀린 걸 다시 확인해봐야겠어. 다음 시험에는 똑같은 실수를 하지 않을 거야.,다음 시험에는 노력한 만큼 좋은 결과가 나와서 원하시는 목표를 성취하시면 좋겠어요.


In [24]:
# 데이터셋 잘 가져왔는지 확인
#print(len(conv_training))
print(len(conv_validation))
#conv_training.sample(n=5)
conv_validation.sample(n=5)

6641


,Unnamed: 0,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템문장1,사람문장2,시스템문장2,사람문장3,시스템문장3
6069,6070,청년,여성,대인관계,해당없음,상처,"가난한, 불우한",회사 사람들이랑 잘 어울리지 못해서 정말 속상해.,회사 사람들이랑 잘 어울리지 못해서 속상하시군요.,같이 회식하러 가자는데 괜찮을지 걱정돼.,왜 걱정이 되시나요?,그동안 재정적으로 어려워서 모임을 갖지 않았던 터라 처음 모임에 참여하는 거거든.,그러셨군요. 그런 상황에 걱정이 되시겠네요.
5124,5125,노년,여성,재정,만성질환 무,당황,고립된,나이는 들었는데 돈이 없어서 집에만 있게 되니 혼자 죽어가는 기분이야.,돈이 없어 홀로 집에서 고립된 기분이시겠어요.,그래. 아들에게 경제적으로 도움을 좀 받을 수 있으면 좋겠는데 내일 연락해 봐야겠어.,그래요. 응원할게요.,NaN,NaN
4018,4019,노년,여성,"건강,죽음",만성질환 유,불안,조심스러운,요실금 때문에 밖에 나가서 누굴 만나지도 못하겠어.,요실금으로 많이 불편하신가 보군요.,혹시 냄새라도 날까 아니면 쳐다볼까 봐 마음 놓고 그냥은 못 나가.,많이 불편하신 게 느껴지네요. 어떻게 해야 지금 상황이 더 나아질 수 있을까요?,요즘에는 시술을 받기고 한다던데 우선 나는 기저귀를 파니깐 사다 써야겠어.,사용자님이 선택하신 대로 진행하시면서 상황이 더 나아지시길 바랄게요.
1171,1172,노년,남성,대인관계,만성질환 유,분노,악의적인,아내한테 자극 좀 주려고 한 건데 그게 지나쳤는지 말을 심하게 했지 뭐야. 그땐 정...,어떻게 말씀하셨는지 좀 더 자세히 말씀해 주시겠어요?,아내가 비만이 되어서 체중 관리를 해야 하는데 나도 모르게 악담을 해버렸어.,체중 관리가 필요한 아내에게 화가 나서 잘못 말씀하셨군요. 아내분과 이후에 대화를 ...,아내한테 미안하다고 해야 할 것 같아. 그렇지만 아내에게 단단히 충고도 해야지.,아내와의 대화로 기분이 풀렸으면 좋겠어요.
6273,6274,청년,여성,"연애,결혼,출산",해당없음,불안,두려운,한평생 가족과 함께 살았는데 이제 내일이면 새로운 가족을 만들어서 지금 사는 집을 ...,새로운 가족이 생기시는 건가요?,응. 맞아. 나 잘할 수 있을까?,그럼요. 잘하실 거라 믿어요.,NaN,NaN


In [25]:
# 감정 레이블을 정수로 변환
conv_training.loc[(conv_training['감정_대분류'] == '기쁨'), '감정_대분류'] = 0
conv_training.loc[(conv_training['감정_대분류'] == '당황'), '감정_대분류'] = 1
conv_training.loc[(conv_training['감정_대분류'] == '분노'), '감정_대분류'] = 2
conv_training.loc[(conv_training['감정_대분류'] == '불안'), '감정_대분류'] = 3
conv_training.loc[(conv_training['감정_대분류'] == '상처'), '감정_대분류'] = 4
conv_training.loc[(conv_training['감정_대분류'] == '슬픔'), '감정_대분류'] = 5

In [26]:
# 필요한 데이터만 data_list 배열에 넣기
data_list = []
for q, label in zip(conv_training['사람문장1'], conv_training['감정_대분류']) :
  data = []
  data.append(q)
  data.append(str(label))

  data_list.append(data)

In [27]:
# data_list에 데이터가 잘 저장되었는지 확인
print(len(data_list))
print(data_list[0])
print(data_list[10000])
print(data_list[20000])
print(data_list[30000])
print(data_list[40000])
print(data_list[50000])
print(data_list[-1])

51630
['퇴사한 지 얼마 안 됐지만 천천히 직장을 구해보려고.', '0']
['간호사들은 돈을 주는 가족들이 없으면 나를 함부로 대하는 것 같은 기분이 들어.', '1']
['요즘따라 짜증이 많이 나.', '2']
['진로에 대해 너무 걱정스러워.', '3']
['오늘 평생 다니던 직장에서 퇴직했어. 오래 다녔는데도 보람이 없고 후회만 되네.', '4']
['부모님이 이혼하셔서 서로 재혼가정이 되었어. 충격이 너무 커서 우울증이 왔어.', '5']
['내가 지금 할 수 있는 건 아무것도 없어. 아내는 계속 아르바이트라도 알아보라고 해.', '5']


In [28]:
# 사이킷런으로 데이터 분리 - train : test = 4 : 1
# from sklearn.model_selection import train_test_split
# dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

# print(len(dataset_train)) # 59604 출력
# print(len(dataset_test)) # 19869 출력

기존 방식 전처리
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [29]:
# 파라미터 설정
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [30]:
# 전처리
tok = tokenizer.tokenize

data_train = BERTDataset(data_list, 0, 1, tok, vocab, max_len, True, False)
# data_test = BERTDataset(dataset_test, 0, 1, tok, vocab, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_list, batch_size=batch_size, num_workers=5)
# test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [31]:
# kobert학습모델
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [32]:
# 정의한 모델 불러오기
# model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

colab python 버전 문제 해결
cuda 재설치
https://velog.io/@kijh30123/No-GPU-errorin-colab 참고




In [33]:
!nvidia-smi
# 런타임-런타임유형-gpu 설정해야함 ^^;

Sun Jan  7 15:50:51 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [34]:
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
!sudo dpkg -i cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
!sudo apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub
!sudo apt-get update
!wget http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64/nvidia-machine-learning-repo-ubuntu1804_1.0.0-1_amd64.deb
!sudo apt install -y ./nvidia-machine-learning-repo-ubuntu1804_1.0.0-1_amd64.deb
!sudo apt-get update

# Install NVIDIA driver
#!sudo apt-get install --no-install-recommends nvidia-driver-418
!sudo apt-get -y installnvidia-driver-418
# Reboot. Check that GPUs are visible using the command: nvidia-smi

# Install development and runtime libraries (~4GB)
#!sudo apt-get install --no-install-recommends \
!sudo apt-get install -y \
    cuda-10-0 \
    libcudnn7=7.6.2.24-1+cuda10.0  \
    libcudnn7-dev=7.6.2.24-1+cuda10.0


# Install TensorRT. Requires that libcudnn7 is installed above.
# !sudo apt-get install -y --no-install-recommends libnvinfer5=5.1.5-1+cuda10.0 \
!sudo apt-get install -y libnvinfer5=5.1.5-1+cuda10.0 \
    libnvinfer-dev=5.1.5-1+cuda10.0

!apt --fix-broken install

--2024-01-07 15:50:51--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2942 (2.9K) [application/x-deb]
Saving to: ‘cuda-repo-ubuntu1804_10.0.130-1_amd64.deb’

cuda-repo-ubuntu180 100%[===================>]   2.87K  --.-KB/s    in 0s      

2024-01-07 15:50:51 (187 MB/s) - ‘cuda-repo-ubuntu1804_10.0.130-1_amd64.deb’ saved [2942/2942]

Selecting previously unselected package cuda-repo-ubuntu1804.
(Reading database ... 121654 files and directories currently installed.)
Preparing to unpack cuda-repo-ubuntu1804_10.0.130-1_amd64.deb ...
Unpacking cuda-repo-ubuntu1804 (10.0.130-1) ...
Setting up cuda-repo-ubuntu1804 (10.0.130-1) ...

The public CUDA GPG key does not appear

In [35]:
# 정의한 모델 불러오기
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [36]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
train_dataloader

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


아래 모델 학습 코드에서

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)): 에서의
ValueError: not enough values to unpack (expected 4, got 2)

- packing : 여러 개의 객체를 하나의 객체로 합치는 것
- unpacking : 여러 개의 객체를 포함하고 있는 하나의 객체를 풀어줌

In [37]:
# 모델 학습
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-37-a7412305aeb3>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/807 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


ValueError: not enough values to unpack (expected 4, got 2)